# Regression

In [2]:
from pathlib import Path
import pandas as pd
import matplotlib.pyplot as plt

df = pd.read_csv("../processed/market_data.csv", parse_dates=["Date"])
df = df.set_index("Date").sort_index()

import statsmodels.api as sm

y = df["btc_logret"]
X = df[["inflation", "fedfunds", "m2_growth", "usdindex_logret", "sp500_logret"]]
X = sm.add_constant(X)

model = sm.OLS(y, X).fit(cov_type="HAC", cov_kwds={"maxlags":3})  # HAC robust SEs
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:             btc_logret   R-squared:                       0.205
Model:                            OLS   Adj. R-squared:                  0.172
Method:                 Least Squares   F-statistic:                     12.75
Date:                Tue, 27 Jan 2026   Prob (F-statistic):           5.65e-10
Time:                        16:14:29   Log-Likelihood:                 58.824
No. Observations:                 128   AIC:                            -105.6
Df Residuals:                     122   BIC:                            -88.53
Df Model:                           5                                         
Covariance Type:                  HAC                                         
===================================================================================
                      coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------------------
const               0.0730      0.037      1.991      0.046       0.001       0.145
inflation         -10.2818      5.562     -1.849      0.065     -21.183       0.619
fedfunds           -0.0083      0.007     -1.174      0.241      -0.022       0.006
m2_growth          -1.0128      1.520     -0.666      0.505      -3.993       1.967
usdindex_logret     1.1569      1.220      0.948      0.343      -1.235       3.549
sp500_logret        2.2707      0.336      6.753      0.000       1.612       2.930
==============================================================================
Omnibus:                       15.287   Durbin-Watson:                   1.330
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               17.991
Skew:                           0.735   Prob(JB):                     0.000124
Kurtosis:                       4.102   Cond. No.                     1.18e+03
==============================================================================

Notes:
[1] Standard Errors are heteroscedasticity and autocorrelation robust (HAC) using 3 lags and without small sample correction
[2] The condition number is large, 1.18e+03. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

# VAR

In [7]:
from statsmodels.tsa.api import VAR
from pathlib import Path
import pandas as pd
import matplotlib.pyplot as plt

df = pd.read_csv("../processed/market_data.csv", parse_dates=["Date"])
df = df.set_index("Date").sort_index()

var_df = df[["btc_logret", "sp500_logret", "usdindex_logret", "inflation", "m2_growth", "fedfunds"]].dropna()

model_var = VAR(var_df)
lag_order = model_var.select_order(maxlags=6)
lag_order.summary()

p = lag_order.selected_orders.get("aic", 2)  # fallback 2
res_var = model_var.fit(p)
res_var.summary()

/Users/kevin/Developer/crypto_macro_analysis/.venv/lib/python3.14/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency ME will be used.
  self._init_dates(dates, freq)


  Summary of Regression Results   
Model:                         VAR
Method:                        OLS
Date:           Tue, 27, Jan, 2026
Time:                     16:18:27
--------------------------------------------------------------------
No. of Equations:         6.00000    BIC:                   -44.3023
Nobs:                     125.000    HQIC:                  -45.8338
Log likelihood:           1979.90    FPE:                4.42313e-21
AIC:                     -46.8817    Det(Omega_mle):     1.89241e-21
--------------------------------------------------------------------
Results for equation btc_logret
                        coefficient       std. error           t-stat            prob
-------------------------------------------------------------------------------------
const                      0.082115         0.047583            1.726           0.084
L1.btc_logret              0.371954         0.104507            3.559           0.000
L1.sp500_logret           -0.313534